In [1]:
# imports and libaries
from dash import Dash, html, dcc, Input, Output, State, no_update
import plotly.express as px
import pandas as pd
from collections import deque
import threading, time, os, math
from datetime import datetime
from threading import Lock

# our device n key auth
DEVICE_ID  = os.environ.get("ARDUINO_DEVICE_ID", "4eba5b6b-d6f4-4d8b-a168-661690be2aa9")
SECRET_KEY = os.environ.get("ARDUINO_SECRET_KEY", "Fb3VV?Nb79rYwGkb@kHHEZkV#")

# samle size and rate of recording
SAMPLE_PERIOD_SEC = 0.5     
N_SAMPLES         = 150     
TITLE = "Accelerometer Live Dashboard"

# gets stored in a file called snapshots
SAVE_DIR = "snapshots"
os.makedirs(SAVE_DIR, exist_ok=True)
last_snapshot_path = None  # updated after each save

# protects our buffers a and b
ingest_lock = Lock()          
plot_lock   = Lock()               

# when buffer is full it saves it and goes onto the next
ingest_A    = []                     


plot_B      = []                  

state_lock  = Lock()
last_vals   = {"x": 0.0, "y": 0.0, "z": 0.0}
last_stamp  = None
last_snapshot_time = None

_stop_event = threading.Event()
_cloud_ok   = False

# IOT cloud
try:
    from arduino_iot_cloud import ArduinoCloudClient
    _lib_ok = True
except Exception as e:
    print("arduino-iot-cloud library missing → SIMULATION mode.\n", e)
    _lib_ok = False

def _cloud_thread_sync():
    """Connect in synchronous mode; update latest x/y/z without async/await in notebooks."""
    global _cloud_ok
    if not _lib_ok or not DEVICE_ID or "PUT_YOUR" in DEVICE_ID:
        print("Cloud credentials not set → SIMULATION mode.")
        return
    try:
        client = ArduinoCloudClient(
            device_id=DEVICE_ID,
            username=DEVICE_ID,         
            password=SECRET_KEY,
            sync_mode=True               
        )

   
        def _set(axis):
            def _inner(*args):
                value = args[-1]
                with state_lock:
                    last_vals[axis] = float(value)
            return _inner

        client.register("py_x", value=0.0, on_write=_set("x"))
        client.register("py_y", value=0.0, on_write=_set("y"))
        client.register("py_z", value=0.0, on_write=_set("z"))

        client.start()
        _cloud_ok = True
        print("Arduino IoT Cloud connected (sync mode). Listening for py_x/py_y/py_z...")

        while not _stop_event.is_set():
            client.update()
            time.sleep(0.02)
    except Exception as e:
        _cloud_ok = False
        print("Cloud connection failed → SIMULATION mode.\nError:", repr(e))

def _simulator_thread():
    """Generate smooth fake data when cloud isn’t configured, so UI is testable."""
    t = 0.0
    while not _stop_event.is_set():
        x = 0.8*math.sin(2*math.pi*0.10*t) + 0.1*math.sin(2*math.pi*0.7*t)
        y = 1.2*math.sin(2*math.pi*0.06*t + 1.1)
        z = 0.6*math.sin(2*math.pi*0.14*t + 2.0)
        with state_lock:
            last_vals.update({"x": x, "y": y, "z": z})
        time.sleep(SAMPLE_PERIOD_SEC)
        t += SAMPLE_PERIOD_SEC

def snapshot():
    """
    Atomically copy EXACTLY N_SAMPLES from Buffer A → Buffer B as a fresh block,
    save it to CSV in SAVE_DIR, then clear Buffer A (no overlap).
    Returns number of samples moved (0 if not ready).
    """
    global last_snapshot_time, last_snapshot_path
    with ingest_lock:
        if len(ingest_A) < N_SAMPLES:
            return 0
        block = list(ingest_A[:N_SAMPLES])  
        ingest_A.clear()                    # resets A for the next block

    # -saving csv
    df_block = pd.DataFrame(block)  # columns: ts, x, y, z
    fname = f"snapshot_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    last_snapshot_path = os.path.join(SAVE_DIR, fname)
    try:
        df_block.to_csv(last_snapshot_path, index=False)
    except Exception as e:
        print("Snapshot save failed:", e)

    # buffer b polish
    with plot_lock:
        plot_B.clear()
        plot_B.extend(block)

    last_snapshot_time = datetime.now()
    return N_SAMPLES

def _sampler_thread():
    """Every 0.5s, snapshot latest x/y/z into Buffer A until it has exactly N; then snapshot()."""
    global last_stamp
    while not _stop_event.is_set():
        with state_lock:
            x, y, z = float(last_vals["x"]), float(last_vals["y"]), float(last_vals["z"])
        now = datetime.now()
        sample = {"ts": now, "x": x, "y": y, "z": z}

        reached_full = False
        with ingest_lock:
            if len(ingest_A) < N_SAMPLES:
                ingest_A.append(sample)
                if len(ingest_A) == N_SAMPLES:
                    reached_full = True
            # else: Buffer A is full; wait for snapshot() to clear it (we skip extras to avoid overlap)

        if reached_full:
            snapshot()  # will clear A and publish to B

        last_stamp = now
        time.sleep(SAMPLE_PERIOD_SEC)

# Starts the threads
cloud_thr = threading.Thread(target=_cloud_thread_sync, daemon=True)
cloud_thr.start()

sampler_thr = threading.Thread(target=_sampler_thread, daemon=True)
sampler_thr.start()

if not _lib_ok or "PUT_YOUR" in DEVICE_ID:
    sim_thr = threading.Thread(target=_simulator_thread, daemon=True)
    sim_thr.start()

# UI 
app = Dash(__name__)

app.layout = html.Div(
    style={"fontFamily": "system-ui, -apple-system, Segoe UI, Roboto, Arial", "padding": "10px"},
    children=[
        html.H2(TITLE),
        html.Div([
            html.Div([
                html.Label("Series"),
                dcc.RadioItems(
                    id="series-choice",
                    options=[
                        {"label": "x", "value": "x"},
                        {"label": "y", "value": "y"},
                        {"label": "z", "value": "z"},
                        {"label": "combined (x,y,z)", "value": "combined"},
                    ],
                    value="combined",
                    inline=True,
                ),
            ], style={"marginRight": "24px"}),

            html.Div([
                html.Label("Chart type"),
                dcc.RadioItems(
                    id="chart-type",
                    options=[{"label": "Line", "value": "line"},
                             {"label": "Scatter", "value": "scatter"}],
                    value="line",
                    inline=True,
                ),
            ]),
        ], style={"display": "flex", "gap": "24px", "flexWrap": "wrap"}),

        html.Div(id="status-line", style={"marginTop": "6px", "opacity": 0.85}),
        dcc.Graph(id="live-graph", style={"height": "520px", "marginTop": "8px"}),

        # Heartbeat; we just *read* Buffer B here. Buffer movements happen in threads.
        dcc.Interval(id="tick", interval=1000, n_intervals=0),
    ]
)

def _df_from_plotB():
    with plot_lock:
        if not plot_B:
            return pd.DataFrame(columns=["ts", "x", "y", "z"])
        return pd.DataFrame(list(plot_B))

@app.callback(
    Output("live-graph", "figure"),
    Output("status-line", "children"),
    Input("tick", "n_intervals"),
    State("series-choice", "value"),
    State("chart-type", "value"),
)
def redraw(_n, series_choice, chart_type):
    # shows the newest block
    df = _df_from_plotB()
    if df.empty:
        fig = px.line(pd.DataFrame({"ts": [], "v": []}), x="ts", y="v", title="Waiting for first block…")
        with ingest_lock:
            a_len = len(ingest_A)
        status = (f"Cloud: {'OK' if _cloud_ok else 'SIM'} • BufferA {a_len}/{N_SAMPLES} • "
                  f"Block size: {N_SAMPLES} • Sample every {SAMPLE_PERIOD_SEC}s")
        return fig, status

    if series_choice == "combined":
        dfl = df.melt(id_vars="ts", value_vars=["x", "y", "z"], var_name="axis", value_name="value")
        fig = (px.line if chart_type == "line" else px.scatter)(
            dfl, x="ts", y="value", color="axis", title="Fresh Block (x,y,z)"
        )
    else:
        title = f"Fresh Block • {series_choice.upper()}"
        fig = (px.line if chart_type == "line" else px.scatter)(df, x="ts", y=series_choice, title=title)

    fig.update_layout(
        margin=dict(l=20, r=20, t=50, b=20),
        xaxis_title="Time", yaxis_title="Value",
        legend_title="Series" if series_choice == "combined" else None,
        hovermode="x unified" if chart_type == "line" else "closest",
    )

    with ingest_lock:
        a_len = len(ingest_A)
    saved_name = os.path.basename(last_snapshot_path) if last_snapshot_path else "—"
    status = (
        f"Cloud: {'OK' if _cloud_ok else 'SIM'} • "
        f"Last snapshot: {last_snapshot_time.strftime('%H:%M:%S') if last_snapshot_time else '—'} • "
        f"File: {saved_name} • BufferA {a_len}/{N_SAMPLES} • "
        f"Block size: {N_SAMPLES} • Sample every {SAMPLE_PERIOD_SEC}s"
    )
    return fig, status


app.run(debug=False, port=8050, jupyter_mode="inline")


def _shutdown():
    _stop_event.set()

Arduino IoT Cloud connected (sync mode). Listening for py_x/py_y/py_z...
